In [1]:
# TCC da Pós-graduação Lato Sensu em Ciência de Dados e Big Data
## Entidade promotora: PUC Minas Virtual
### Trabalho de Conclusão de Curso apresentado ao Curso de Especialização em Ciência de Dados e Big Data como requisito parcial à obtenção do título de especialista.
### Projeto: Preditores de óbitos por COVID-19: O aprendizado de máquina como instrumento auxiliar na definição de políticas públicas

### Aluno: Breno Marques Barreto

# Considerações importantes sobre a utilização dos modelos salvos:
# Deve-se conferir se a versão do Python que irá utilizar é a mesma usada neste projeto.
# Aconselha-se utilizar para deserializar o modelo salvo:
#   1 - A mesma versão do Python. 
#   2 - As mesmas versãoes das bibliotecas usadas no projeto. 
# As orientações não são apenas para as versões da NumPy e da scikit-learn.

# BIBLIOTECAS

In [2]:
# Bibliotecas
import numpy as np
import pandas as pd
import datetime
import os.path
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
seed=45
%matplotlib inline
plt.style.use('fivethirtyeight')

# CARGA DOS DADOS

In [3]:
#################################################################
# Esta Função carrega os dados de um arquivo csv para um dataframe 
################################################################# 
def load_df(filename,psep,pusecols=None,pconverters=None):
    """Esta carrega os dados de um arquivo csv para um dataframe.
          pusecols - limita as colunas a serem carregadas.
          pconverters - define a forma de conversão de dados"""
    usecols = pusecols
    converters = pconverters
    df = pd.read_csv(filename, sep=psep, usecols=pusecols, converters=pconverters)
    return df

# PROCESSAMENTO

In [4]:
variavel_alvo = ['OBT_COVID']

In [5]:
#################################################################
# Função para converter os tipos de dados não identificados pelo pandas e otimizar o processamento 
#################################################################
def conv(val):
    """Função para converter os tipos de dados não identificados pelo pandas e otimizar o processamento."""
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)

In [6]:
#################################################################
# Esta função realiza a codificação dos dados 
#################################################################
def coding(col, codeDict):
  """Esta função realiza a codificação dos dados."""
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

In [7]:
#################################################################
# Esta função seleciona as variáveis 
# conforme o nível mínimo de variância estipulado por threshold
#################################################################
def elimina_baixa_variancia(X_data,threshold=0.1):
    """Esta função Seleciona, no dataset de treino,
       as variáveis conforme o nível mínimo de variância estipulado por threshold."""
    print("Num. variáveis inicial:{}".format(len(X_data.columns)))
    selector = VarianceThreshold(threshold)
    selector.fit(X_data)
    print("Num. variáveis final:{}".format(len(X_data.columns)))    
    return X_data[X_data.columns[selector.get_support(indices=True)]]


In [8]:
#################################################################
# Esta função corrige idades com valores negativos
#################################################################
def corrige_idade(covid_df):
    """Esta função corrige idades com valores negativos."""
    # Há duas abordagens: substituir pela média ou pelo valor absoluto. 
    # Esta hipótese pode gerar distorções se houver valores muito grandes.
    # Optou-se pela média
    covid_df.NU_IDADE_N [covid_df.NU_IDADE_N<0] = covid_df.NU_IDADE_N.mean()
    return covid_df

In [9]:
#################################################################
# Esta função cria uma variável categórica baseada na idade (NU_IDADE_N)
#################################################################
def cria_faixa_etaria(covid_df):
    """Esta função cria variável categórica baseada na idade (NU_IDADE_N)"""
    pbins=[0, 20, 29, 39, 49, 59, 69, 79, sys.maxsize]
    plabels=['0-20', '20-29', '29-39', '39-49','49-59', '59-69','69-79','79-150']
    covid_df['NU_IDADE_CAT'] = pd.cut(covid_df['NU_IDADE_N'], bins=pbins, labels=plabels, include_lowest=True) 
    codigos_idade = {'0-20':0, '20-29':1, '29-39':2, '39-49':3,'49-59':4, '59-69':5,'69-79':6,'79-150':7}
    covid_df['NU_IDADE_CAT'] = coding(covid_df['NU_IDADE_CAT'], codigos_idade).astype(int)  
    return covid_df

In [10]:
#################################################################
# Esta função une os dois datasets através das colunas CO_MUN_NOT
#################################################################
def merge_dfs(geoses_df, covid_df):
    """Esta função une os dois datasets através das colunas CO_MUN_NOT"""
    # Renomeia a coluna de geoses_df que servirá para o merge
    geoses_df.rename(columns={'v0002_codigo_do_municipio': 'CO_MUN_NOT'}, inplace = True)
    # Seleciona apenas as colunas relevantes
    columns = ['CO_MUN_NOT','GeoSES']
    geoses_df = geoses_df[columns]

    # As tabelas geoses_df e covid_df serão unidas (merge) através do campo "CO_MUN_NOT",
    # que possui formatos diferentes nelas. Em geoses_df possui 7 dígitos,
    # como nas tabelas do IBGE, e 6 dígitos em covid_df.
    # O último dígito não é, aparentemente, significante. 
    # Decidiu-se retirá-lo daquela tabela e adequar o campo ao tamanho da tabela covid_df, 6 dígitos
    geoses_df['CO_MUN_NOT'] = geoses_df['CO_MUN_NOT'].apply(lambda x: str(x)[:-1]).astype(int)

    # Confirmar que não há registros duplicados em geoses_df, garantindo-se a integridade.
    print("Total de registros em geoses_df:",len(geoses_df.groupby ('CO_MUN_NOT').CO_MUN_NOT.count()))

    # Une os dois dataframes através do campo CO_MUN_NOT
    merged_df = covid_df.merge(geoses_df, on='CO_MUN_NOT')

    # Exclusão da variavel "CO_MUN_NOT". 
    merged_df = merged_df.drop(['CO_MUN_NOT'], axis = 1)
    return merged_df

In [11]:
#######################################################################################
# Esta função define a variável alvo como decorrência de 'CLASSI_FIN' e 'EVOLUCAO'
#######################################################################################
def define_variavel_alvo(covid_df):
    """Esta função define a variável alvo como decorrência de 'CLASSI_FIN' e 'EVOLUCAO' """
    # CLASSI_FIN COVID e EVOLUCAO para óbito será a variavel a ser predita
    condicao=[(covid_df['CLASSI_FIN']==5.0) & (covid_df['EVOLUCAO']==2.0), (covid_df['EVOLUCAO']!=2.0)]
    resultados = [1, 0]
    # Se o registro foi classificado como COVID e evoliu para óbito, OBT_COVID será preenchida como 1, caso contrário 0.
    covid_df['OBT_COVID']=np.select(condicao, resultados)
    # Exclusão da variavel "EVOLUCAO". Não pode ser mantida pois resultaria em acurácia de 100%.
    # Foi substituída por OBT_COVID.
    covid_df=covid_df.drop(['EVOLUCAO'], axis = 1)
    return covid_df

In [12]:
#######################################################################################
# Esta Função converte os tipos das colunas com até 10 valores distintos em categoricas
#######################################################################################
def converte_tipo_category(covid_df,list_cols=None):
    """Esta função converte o tipo das colunas com até 10 valores distintos em categorica"""
    if (list_cols is None):
        col = covid_df.columns
    else:
        col = list_cols
    covid_df_temp = covid_df.copy()    
    for i in col:
        if covid_df[i].nunique()<=10:     #Filtra pelo número de ocorrências as que serão convertidas
            covid_df_temp[i] = covid_df[i].astype('category')
    return covid_df_temp


### NORMALIZAÇÃO DOS DADOS

In [13]:
from sklearn.preprocessing import MinMaxScaler # importa a biblioteca para execução da normalização/reescala
from sklearn.preprocessing import StandardScaler
########################################################
# Esta função realiza a reescala dos atributos contínuos
########################################################
def normaliza_dados(pcovid_df,list_cols=None):
    """Esta função realiza a reescala dos atributos contínuos"""

    covid_df = pcovid_df.copy()
    if (list_cols is None):
        num_col = covid_df.select_dtypes(include =['float64','int64','int32']).columns
    else:
        num_col = list_cols
    #gera uma cópia do dataframe
    covid_df_adjusted_MinMaxScaler = covid_df.copy()  
    #executa a reescala/normalização
    covid_df_adjusted_MinMaxScaler[num_col] = MinMaxScaler().fit_transform(covid_df_adjusted_MinMaxScaler[num_col]) 
    return covid_df_adjusted_MinMaxScaler

### ANÁLISE DA CORRELAÇÃO

In [14]:
#######################################################################################
# Esta função calcula a correlação entre as variáveis do dataframe com a variável 
# alvo e retornar as com correlação maior que o parâmetro escolhido 
#######################################################################################
def verifica_correlacao(covid_df,grafico=False,parametro=0.01):
    """Esta função calcula a correlação entre as variáveis do daataframe
       e rotorna as que estão acia de determinado valor"""
    # Correlacao com a variavel alvo usando a Correlação de Pearson 
    plt.figure(figsize=(30,25))
    cor = covid_df.corr()
    sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
    if (grafico == True):
        plt.show()
    cor_target = abs(cor['OBT_COVID'])
    #Selecionadas as variáveis com maior correlação
    relevant_features = cor_target[cor_target>parametro].sort_values(ascending=False)
    return relevant_features

### CONVERSÃO DAS VARIÁVEIS CATEGÓRICAS EM BINÁRIAS

In [15]:
import sys
####################################################################################
# Esta função converte os atributos categóricos para binários, exceto a variável alvo.
####################################################################################
def converte_binarios(covid_df,list_cols=None):
    """Esta função converte os atributos categóricos para binários, exceto a variável alvo"""
    if (list_cols is None):
        # Seleciona todas as variáveis categóricas do dataframe exceto a alvo.
        cat_col = covid_df.select_dtypes(include =['category']).columns.difference(['OBT_COVID']).values 
    else:
        # Seleciona todas as colunas recebidas
        cat_col = list_cols
    # Converte as variáveis categóricas em binárias
    covid_df_bin = pd.get_dummies(covid_df, columns=cat_col)  # cat_col contém os atributos categóricos.
    # Reordena a coluna "OBT_COVID" para ser a ultima, facilitando a separação da variável alvo
    cols = [col for col in covid_df_bin if col != 'OBT_COVID'] + ['OBT_COVID']
    covid_df_bin = covid_df_bin[cols]
    return covid_df_bin

In [16]:
####################################################################################
#Esta função exibe as ocorrências de valores únicos em cada coluna da dataframe
####################################################################################
def unicos(covid_df):
    """Esta função exibe as ocorrências de valores únicos em cada coluna da dataframe"""
    col = covid_df.columns
    k = pd.DataFrame(index=col)
    for i in col:
        k['No de únicos'] = covid_df[i].nunique()
        k['Prim. valor unico'] = covid_df[i].unique()[0]
        k['Seg. valor unico'] = covid_df[i].unique()[1]
        return k.T

### BALANCEAMENTO DA BASE DE DADOS

In [17]:
from sklearn.datasets import make_classification
from collections import Counter
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.over_sampling import SMOTENC

In [18]:
from pathlib import Path
##############################################################################
# Esta função realiza o balanceamento das classes alvo através do método SMOTE
##############################################################################
def imblearn_SMOTE_oversampling(covid_df, filename=None, reprocessa=False, ptarget='OBT_COVID'):
    """Esta função realiza o balanceamento das classes alvo através do método SMOTE
       Parametros: covid_df: dataframe desbalanceado e string com nome da variável alvo
                   filename: nome do arquivo antes do processo de oversampling
                   reprocessa: se o dataframe deve ser reprocessado mesmo que o processado exista
                   ptarget : nome da variável alvo
      Retorno: Dataframe balanceado através do SMOTE da biblioteca imblearn
      Se já existe o arquivo processado, apenas carrega
    """
    numfeatures = str(covid_df.shape[1]) # será usado na formacao do nome do arquivo csv
    if (filename is None):  # se não foi passado nome de arquivo, efetua o balanceamento do dataframe
        print("Nome de arquivo não informado.")
        print("Aguarde, efetuando processamentofor do dataframe...")
        print(covid_df[ptarget].value_counts())
        X = covid_df.drop(ptarget, axis=1)
        Y = covid_df[ptarget]
        sm = SMOTE(random_state=42)
        X_res, Y_res = sm.fit_resample(X, Y)
        df_smote_over = pd.concat([pd.DataFrame(X_res), pd.DataFrame(Y_res, columns=[ptarget])], axis=1)
        print('SMOTE over-sampling:')
        print(df_smote_over[ptarget].value_counts())
        #salva o arquivo balanceado
        filename_pre ="covid_df" + numfeatures + "_balanceado.csv"
        df_smote_over.to_csv(filename_pre,index = False, sep=';')
    else:  # foi passado o nome de um arquivo da base desbalanceada
            os.path.isfile(filename)
            obj_filename = Path(filename)
            print("Arquivo informado:" ,filename)
            #monta o nome do arquivo da base balanceada
            ext_file = "_proc_smote_over.csv"
            rad_filename = filename[:-4]  #exclui a extensão do arquivo
            filename_pre = rad_filename + "_" + numfeatures + ext_file
            if obj_filename.is_file():  # se o arquivo da base não balanceada existe
                #verifica se o arquivo processado existe
                os.path.isfile(filename_pre)
                file_filename_pre = Path(filename_pre)
                print("Verifica existencia do arquivo:" ,file_filename_pre.is_file())
                if file_filename_pre.is_file() and reprocessa==False: # se existe e não reprocessar
                    #carrega o arquivo csv já balanceado
                    print("Arquivo {} foi encontrado.".format(file_filename_pre))
                    print("Carregando o arquivo...")
                    df_smote_over = load_df(file_filename_pre,";",None,None)
                    print("Concluído.")
                else:  # o arquivo preprocessado não existe ou o balanceamento deve ser refeito
                    #Faz o balanceamento dos dados
                    print("Arquivo {} não encontrado.".format(file_filename_pre))
                    print("Aguarde, efetuando balanceamento das classes da variável alvo...")
                    print(covid_df[ptarget].value_counts())
                    X = covid_df.drop(ptarget, axis=1)
                    Y = covid_df[ptarget]
                    sm = SMOTE(random_state=42)
                    X_res, Y_res = sm.fit_resample(X, Y)
                    df_smote_over = pd.concat([pd.DataFrame(X_res),pd.DataFrame(Y_res,columns=[ptarget])],axis=1)
                    print('SMOTE over-sampling:')
                    print(df_smote_over[ptarget].value_counts())
                    #salva o arquivo balanceado
                    df_smote_over.to_csv(filename_pre,index = False, sep=';')
            else:  #O arquivo da base passada como nao balanceada nao existe
                print("O arquivo {} referenciado como base desabalanceada não existe.".format(filename))
                print("Forneça um nove de arquivo válido.")
    #Exibe gráfico com as classes balanceadas
    df_smote_over[ptarget].value_counts().plot(kind='bar', title='Classes OBT_COVID') 
    return df_smote_over 


# CRIAÇÃO DOS MODELOS

In [19]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import imblearn
from sklearn.datasets import make_classification
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split

In [20]:
####################################################################################
# Esta função cria uma matriz de correlação com as variáveis do dataframe
####################################################################################
def matrix_correlacao(covid_df,pbest_feat=None, ptarget='OBT_COVID'):
    """Esta função cria uma matriz de correlação com as variáveis do dataframe.   
    """
    if (pbest_feat is None):
        best_feat = covid_df.columns
    else:
        best_feat = pbest_feat + [ptarget] 
    print("Melhores features",best_feat)
    X = covid_df.iloc[:,:-1]  #variáveis independentes
    y = covid_df.iloc[:,-1].astype(int) #variável alvo, última coluna. Converte para int devido a SelectKBest
    #Obtém as correlações de cada variavel no dataset
    corrmat = covid_df[best_feat].corr()
    #print(corrmat)
    top_corr_features = corrmat.index
    plt.figure(figsize=(35,35))
    #plot heat map
    g=sns.heatmap(covid_df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [21]:
###########################################################
# Estas funções serializam e deserializam os modelos criados 
############################################################
from numpy import loadtxt
import pickle

def save_model(pmodel, pmodel_name):
    """Esta função salva o modelo."""
    pickle.dump(pmodel, open(pmodel_name, "wb"))
    return

def load_model(pmodel_name):
    """Esta função carrega um modelo."""
    pickle.load(open(pmodel_name, "rb"))
    return

In [22]:
###########################################################
#      Esta função recebe o dataset de treino 
#       e um tipo de classificador. O treina
#       com os hiperparâmetros já ajustados 
#       e retorna o modelo treinado 
###########################################################
def cria_modelo(X_train,y_train,classificador,tuned=False):
    """Esta função recebe o dataset de treino 
       e um tipo de classificador. Dependendo do
       valor do parametro tuned, cria e treina
       com os hiperparâmetros já ajustados 
       e o retorna.   
    """

    if classificador=="1":
        # XGBClassifier
        # Ajustados com HalvingRandomSearchCV no notebook “COVID_Ajustar_XGB.iypnb".
        params_XGB = {
                'subsample': 0.8,
                'reg_lambda': 50,
                'min_child_weight': 3,
                'max_depth': 5,
                'learning_rate': 0.5,
                'gamma': 5,
                'colsample_bytree': 0.8}
        if (tuned == True):
            modelo = XGBClassifier(**params_XGB)
            print("Selecionado modelo XGBClassifier Ajustado.")
        else:
            modelo = XGBClassifier()
            print("Selecionado modelo XGBClassifier padrão.")
    elif classificador=="2":
        # LogisticRegression
        # Parametros obtidos através do notebook "COVID_Ajustar_Logistic_Regression.ipynb"
        params_lr = {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
        if (tuned == True):
            modelo = LogisticRegression(**params_lr)
            print("Selecionado modelo LogisticRegression Ajustado.")
        else:
            modelo = LogisticRegression()
            print("Selecionado modelo LogisticRegression padrão.")
    elif classificador == "3":  
        # KNN
        modelo = KNeighborsClassifier(n_neighbors=4)
        print("Selecionado modelo KNN.") 
    elif classificador == "4":
        # RANDOM FOREST
        modelo = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42).fit(X_train, np.ravel(y_train))
        print("Selecionado modelo RANDOM FOREST.")
    elif classificador == "5":    
        # DecisionTree
        # Parametros obtidos através do notebook "COVID_Ajustar_DecisionTree.ipynb"
        params_DecisionTree = {'ccp_alpha':0.0,
                'class_weight':None,
                'criterion':'gini',
                'max_depth':15,
                'max_features':None,
                'max_leaf_nodes':None,
                'min_impurity_decrease':0.0,
                'min_impurity_split':None,
                'min_samples_leaf':1, #padrao
                'min_samples_split':2, #padrao
                'min_weight_fraction_leaf':0.0,
                'presort':'deprecated',
                'random_state':42,
                'splitter':'best'
        }
        if (tuned == True):      
            modelo =  DecisionTreeClassifier(**params_DecisionTree)
            print("Selecionado modelo DecisionTreeClassifier ajustado.")             
        else:
            modelo =  DecisionTreeClassifier()            
            print("Selecionado modelo DecisionTreeClassifier padrão.")       
    elif classificador == "6":  
        #Importa o modelo svm 
        from sklearn import svm
        #Cria o classficador svm
        modelo = svm.SVC(kernel='linear') # Linear Kernel
    elif classificador == "7": 
        #ExtraTreesClassifier
        if (tuned == True):    
            #Parametros obtidos através do notebook "COVID_Ajustar_ExtraTreesClassifier.ipynb"     
            params_XTC = {'criterion': 'entropy',
                'max_depth': 55,
                'max_features': 40,
                'min_samples_split': 16,
                'n_estimators': 300}

            modelo = ExtraTreesClassifier(**params_XTC)
            print("Selecionado modelo ExtraTreesClassifier ajustado.")
        else:
            modelo = ExtraTreesClassifier()
            print("Selecionado modelo ExtraTreesClassifier padrão.")  
    else:
        print ("Modelo inválido.")
        return
    #Treina o modelo usando os dados de treinamento recebidos como parâmetros.
    modelo.fit(X_train, y_train)  # treino com dados sem balanceamento

    return modelo

In [23]:
from sklearn.metrics import plot_confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import mean_squared_error
###########################################################
# Esta função avalia a performance de um modelo
############################################################
def avalia_modelo(X_test,y_test,modelo,rank_score, id):
    """Esta função avalia a performance de um modelo. Parâmetros: X_test,y_test,modelo,rank_score, id """
    y_pred = modelo.predict(X_test)
    plot_confusion_matrix(modelo, X_test, y_test,cmap=plt.cm.Blues, values_format='d')
    plt.ylabel('Verdadeiro')
    plt.xlabel('Previsto')
    plt.xticks([0, 1], ['Não','Sim'])
    plt.yticks([0, 1], ['Não','Sim'])

    # Exibição de métricas de classificação:
    model_name = type(modelo).__name__
    indice = model_name + id 
    y_pred = modelo.predict(X_test)
    rank_score[indice] = classification_report(y_test, y_pred)
    print(rank_score[indice])
    return rank_score

### FEATURE SELECTION

In [24]:

####################################################################################
# Esta função identifica, através de "Eliminação Recursiva de Variáveis" (RFE), o número ótimo de variáveis
####################################################################################
from sklearn.feature_selection import RFE
def obtem_num_features(covid_df_pre_dummies, modelo):
    """Esta função identifica, através de "Eliminação Recursiva de Variáveis" (RFE), o número ótimo de variáveis.
      Parâmetros: covid_df_pre_dummies, modelo"""   
    labels = covid_df_pre_dummies.columns[:-1]  #última coluna comtém o atributo dependente (OBT_COVID)
    X = covid_df_pre_dummies[labels] #dataframe com todas as colunas exceto a variável alvo
    y = covid_df_pre_dummies['OBT_COVID'].astype('category') #ndarray com a variável a ser classificada.
      #no de features
    num_max = covid_df_pre_dummies.shape[1]
    nof_list=np.arange(1,num_max)            
    high_score=0
    #Variavel para armazenar o numero otimo de features
    nof=0           
    score_list =[]
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 0)
        model = modelo
        rfe = RFE(model,nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)
        score = model.score(X_test_rfe,y_test)
        score_list.append(score)
        if(score>high_score):
            high_score = score
            nof = nof_list[n]
    print("Num. ótimo de variáveis: %d" %nof)
    print("Score com %d variáveis: %f" % (nof, high_score))
    return nof

In [25]:
####################################################################################
# Esta função utiliza "Eliminação Recursiva de Variáveis" (RFE) para identificar as 
# melhores variáveis
####################################################################################
def sel_features(covid_df_pre_dummies,modelo, num_feat):
    """Esta função utiliza "Eliminação Recursiva de Variáveis" (RFE) para identificar as 
       melhores variáveis.
       Parâmetros: covid_df_pre_dummies,modelo, num_feat
       """
    #Metodo RFE para eliminação recursiva de variáveis
    from sklearn.feature_selection import RFE
    labels = covid_df_pre_dummies.columns[:-1]  #última coluna comtém o atributo dependente (OBT_COVID)
    X = covid_df_pre_dummies[labels] #dataframe com todas as colunas exceto a variável alvo
    y = covid_df_pre_dummies['OBT_COVID'].astype('category') #ndarray com a variável a ser classificada.
    #Inicializa o modelo RFE
    rfe = RFE(modelo, num_feat)
    #Transforma os dados usando RFE
    X_rfe = rfe.fit_transform(X,y)  
    #Treina o modelo
    modelo.fit(X_rfe,y)
    print(rfe.support_)
    print(rfe.ranking_)
    return rfe.ranking_

In [26]:
####################################################################################
#Esta função exibe as ocorrências de valores únicos em cada coluna da dataframe
####################################################################################
def sel_features_importance(covid_df,model,k_best=10):
    """Esta função seleciona as "melhores" variáveis para o modelo passado como parametro.
    Parâmetros: covid_df,model,k_best=10 """
    X = covid_df.iloc[:,:-1]  #variáveis independentes
    y = covid_df.iloc[:,-1].astype(int) #variável alvo, última coluna. Converte para int devido a SelectKBest
    model.fit(X,y)
    #Monta o gráfico para melhor visualização da importância das variáveis 
    feat_importance = pd.Series(model.feature_importances_, index=X.columns)
    feat_importance.nlargest(k_best).plot(kind='barh')  #prepara o grafico
    plt.show() #exibe o grafico
    list_best = feat_importance.nlargest(k_best).index.tolist() #retorna os nomes das variáveis em forma de lista
    return list_best

In [27]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
####################################################################################
# Esta função seleciona, genericamente, as variáveis com maior correlação com a variável dependente. 
####################################################################################
def sel_univariate_features(covid_df,k_best=10):
    """Esta função seleciona, genericamente, as variáveis com maior correlação com a variável dependente.
       Parâmetros: covid_df,k_best"""
    #garante a normalizacao dos dados, pressuposto de SelectKBest
    covid_df= normaliza_dados(covid_df)
    #presume-se que a última variável (-1) do dataframe é a alvo
    labels = covid_df.columns[:-1]  
    col_cat = []  #armazenará os índices
    for val in labels:
        idx_col = covid_df.columns.get_loc(val) #obtém os respectivos índices
        col_cat.append(idx_col) 

    X = covid_df.iloc[:,:-1]  #variáveis independentes
    y = covid_df.iloc[:,-1].astype(int) #variável alvo, última coluna. Converte para int devido a SelectKBest

    #Aplica a classe SelectKBest para extrair as "k_best" melhores variáveis
    bestfeatures = SelectKBest(score_func=chi2, k=k_best)
    fit = bestfeatures.fit(X,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    #concat os dois dataframes para melhor visualização 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['variavel','Score']  #nomeaia as colunas do dataframe 
    print(featureScores.nlargest(k_best,'Score'))  #imprime as k_best melhores
    lista_best = featureScores.nlargest(k_best,'Score')['variavel'].tolist() # prepara um ndarray das melhores variáveis para retornar
    return lista_best

In [28]:
####################################################################################
# Esta função seleciona as melhores variáveis para o modelo passado como parametro. 
####################################################################################
def sel_lim_features_importance(covid_df,model,lim=0.01):
    """Esta função seleciona as melhores variáveis para o modelo passado como parametro """
    X = covid_df.iloc[:,:-1]  #variáveis independentes
    y = covid_df.iloc[:,-1].astype(int) #variável alvo, última coluna. Converte para int devido a SelectKBest
    model.fit(X,y)
    #Monta o gráfico para melhor visualização da importância das variáveis 
    feat_importance = pd.Series(model.feature_importances_, index=X.columns)
    indice = []
    for i in range(len(feat_importance)):
        if feat_importance[i] >= lim:
            indice.append(i)
    return indice